In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc
import requests
from urllib.parse import urlparse

In [2]:
# input data: 주소, 건물 정보. 토지 정보는 자동으로 가져와야 함. 토지 정보를 자동으로 가져오려면 오리지널 토지특성정보 &
# 토지이용계획정보, 좌표는 api에서 다운로드해야함.

In [3]:
# 1. 오리지널 토지특성정보, 토지이용계획정보 & 공시지가
# 2. 좌표
# 3. 건물 정보

In [4]:
# 1. 오리지널 토지특성정보 & 토지이용계획정보

In [5]:
p_land_specs_df = pd.read_csv('./토지특성정보/land_specs_production_ver_1.csv')

In [6]:
print(p_land_specs_df.shape)
p_land_specs_df.head()

(983625, 9)


,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지번주소
0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,서울특별시 강남구 개포동 100-0
1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 101-0
2,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 102-0
3,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,서울특별시 강남구 개포동 103-0
4,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,서울특별시 강남구 개포동 104-1


In [7]:
p_land_plans_df = pd.read_csv('./토지이용계획정보/seoul_land_plans_production_ver_1.csv')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

In [8]:
print(p_land_plans_df.shape)
p_land_plans_df.head()

(983564, 350)


,지번주소,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,4대문안,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,공항시설보호지구,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기업형임대주택 공급촉진지구,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보조간선도로,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교시설보호지구,학교이적지,학교환경위생 정화구역,현상변경허가 대상구역,홍수관리구역,화장장,환경정비구역
0,서울특별시 강남구 개포동 100-0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,접함,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,저촉,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
p_land_plans_df = p_land_plans_df.fillna('해당사항없음')
p_land_plans_df.head()

,지번주소,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,4대문안,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,공항시설보호지구,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기업형임대주택 공급촉진지구,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보조간선도로,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교시설보호지구,학교이적지,학교환경위생 정화구역,현상변경허가 대상구역,홍수관리구역,화장장,환경정비구역
0,서울특별시 강남구 개포동 100-0,2020,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당

In [10]:
p_land_prices_df = pd.read_csv('./공시지가/land_prices_production_ver.csv')

In [11]:
print(p_land_prices_df.shape)
p_land_prices_df.head()

(903493, 5)


,지번주소,특수지구분명,기준년도,공시지가(만원),표준지여부
0,서울특별시 강남구 개포동 100-0,일반,2020,39.60,N
1,서울특별시 강남구 개포동 101-0,일반,2020,47.52,N
2,서울특별시 강남구 개포동 102-0,일반,2020,47.52,N
3,서울특별시 강남구 개포동 103-0,일반,2020,30.97,N
4,서울특별시 강남구 개포동 104-1,일반,2020,39.46,N


In [12]:
# 2. 좌표

In [13]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):        
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1'\
                                       + '&keyword=' + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        # 지번주소에 매칭하는 도로명주소가 없을 경우, 근처 지번주소의 도로명주소로 대체
        
        if roadname_info == []:
            splitted = address_no_zero.split('-')
            space_splitted = address_no_zero.split(' ')
            
            if len(splitted) == 1:
                bonbun = int(space_splitted[3])
                bonbun_positive = bonbun
                bonbun_negative = bonbun
                
                while roadname_info == []:
                
                    bonbun_positive += 1
                    bonbun_negative -= 1
                    
                    addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                    + str(bonbun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                        + str(bonbun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                
                
            else:
                bubun = int(splitted[1])
                bubun_positive = bubun
                bubun_negative = bubun
                
                count = 0
                while roadname_info == []:
                    
                    bubun_positive += 1
                    bubun_negative -= 1
                    
                    addr = splitted[0] + '-' + str(bubun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = splitted[0] + '-' + str(bubun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                    
                    count += 1
                    
                    if count > 10:
                        
                        addr = splitted[0]
                        
                        space_splitted = addr.split(' ')
                        
                        bonbun = int(space_splitted[3])
                        bonbun_positive = bonbun
                        bonbun_negative = bonbun
                        
                        while roadname_info == []:

                            bonbun_positive += 1
                            bonbun_negative -= 1

                            addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                            + str(bonbun_positive)
                            
                            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                           + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                           + '&hstryYn=Y')
                            roadname_info = roadname_result.json()['results']['juso']

                            if roadname_info == []:

                                addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                                + str(bonbun_negative)
                                
                                roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey\
                                                               + '&currentPage=1' + '&countPerPage=1' + '&keyword='\
                                                               + addr + '&resultType=json' + '&hstryYn=Y')
                                roadname_info = roadname_result.json()['results']['juso']
                                
        roadname_info = roadname_info[0]                                
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [14]:
p_land_specs_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_specs_df.columns]

In [15]:
p_land_specs_df.columns

Index(['지목명', '토지면적', '용도지역명1', '용도지역명2', '토지이동상황', '지형높이', '지형형상', '도로접면',
       '지번주소'],
      dtype='object')

In [16]:
p_land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_plans_df.columns]

In [17]:
p_land_plans_df.columns

Index(['지번주소', '년', '(한강)오염행위_제한지역', '(한강)폐기물매립시설_설치제한지역', '4대문안',
       '가로구역별_최고높이_제한지역', '가스공급설비', '가스공급시설', '가축사육제한구역', '개발제한구역',
       ...
       '하천구역', '하천미분류', '학교', '학교시설보호지구', '학교이적지', '학교환경위생_정화구역',
       '현상변경허가_대상구역', '홍수관리구역', '화장장', '환경정비구역'],
      dtype='object', length=350)

In [18]:
#manual_input_features = ['지번주소', 'trade_type', '총연면적', '전용면적', '층mean', '주건축물수', '총건축면적', '호수대비연면적', '옥내자주식대수', '호수대비부속건축물면적', '총호수', '층', '호수대비주차수']

In [19]:
#features_to_fill_df = pd.DataFrame({'features':manual_input_features})

In [20]:
#features_to_fill_df.to_excel('Production_Prep/직접작성필요목록.xlsx', index=False)

In [21]:
training_data_chunk = next(pd.read_csv('./Training/training_data_ver_17/training_data_ver_17.csv', chunksize=1))

In [22]:
print(training_data_chunk.shape)
training_data_chunk.head()

(1, 310)


,전용면적,금액(만원),층,건축년도,가격면적,target_log_transformed,targetarea_log_transformed,지번주소,주택가격지수,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type,공시지가(만원),표준지여부,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,(한강)오염행위_제한지역,(한강)폐기물매립시설_설치제한지역,4대문안,가로구역별_최고높이_제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축허가·착공제한지역,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택_공급촉진지구,공공청사,공동구,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,공항시설보호지구,과밀억제권역,광로1류(폭_70M_이상),광로2류(폭_50M~70M),광로3류(폭_40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭_35M~40M),대로2류(폭_30M~35M),대로3류(폭_25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획_입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,묘지공원,문화공원,문화시설,문화재,문화재보존영향_검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,방송통신시설,방수설비,방재지구,방화지구,변전소(전원개발사업구역),변전시설,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,소공원,소로1류(폭_10M~12M),소로2류(폭_8M~10M),소로3류(폭_8M_미만),소하천,소하천구역,수도공급시설,수평표면구역,시가지경관지구,시가지조성사업지역,시장,시장정비구역,시장정비구역기타,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,운동장,원추표면구역,원형보존지,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차전용도로,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종_구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(전술항공5km),제한보호구역(후방지역500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거용지,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭_20M~25M),중로2류(폭_15M~20M),중로3류(폭_12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역,통제보호구역(민통선이남300m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,학교,학교시설보호지구,학교이적지,학교환경위생_정화구역,현상변경허가_대상구역,홍수관리구역,target_log
0,77.75,57000.0,2,1988,733.118971,10.950807,6.597308,서울특별시 강남구 개포동 655-2,79.1,7.666667,-0.866025,-0.5,5579.095707,-9048.956121,1.556315e+07,4.094180e+07,-0.364267,-0.920469,0.931294,0.390815,-5.048499e+07,0,921.0,0,0,1168.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.597308


In [23]:
cols_to_drop = ['target_log_transformed', 'targetarea_log_transformed', '지번주소', '주택가격지수', '금액(만원)', 'target_log', '전용면적', '층', '토지면적']

In [24]:
training_features = training_data_chunk.columns.drop(cols_to_drop + ['가격면적'])

In [25]:
import lightgbm as lgb

In [26]:
basedir = './Training/training_data_ver_17/'

In [27]:
model_names_list = [f for f in os.listdir(basedir) if 'no_landarea' in f]
model_names_list

['lightgbm_mse_no_landarea_0.txt',
 'lightgbm_mse_no_landarea_1.txt',
 'lightgbm_mse_no_landarea_2.txt',
 'lightgbm_mse_no_landarea_3.txt',
 'lightgbm_mse_no_landarea_4.txt']

In [28]:
models_list = []
for f in tqdm(model_names_list):
    model = lgb.Booster(model_file=basedir + f)
    models_list.append(model)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.29it/s]


In [29]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [30]:
def datetime_feature_engi(df):
    df = df.copy()
    
    current_date = datetime.datetime.now()
    
    year = 2020 #current_date.year
    month = 10#current_date.month
    
    df['year_linear'] = (df['건축년도'] - 2006) + ((month-1)/12)
    
    df['month_sin'] = month_circular_sine_func(month)
    df['month_cos'] = month_circular_cosine_func(month)
    
    return df

In [31]:
def coor_feature_engi(coor_df, coor_minmax_df):
    coor_df = coor_df.copy()
    coor_minmax_df = coor_minmax_df.copy()
    
    coor_df['xnorm'] = (coor_df['x좌표'] - coor_minmax_df['xmin'] - ((coor_minmax_df['xmax'] - coor_minmax_df['xmin'])/2))
    coor_df['ynorm'] = (coor_df['y좌표'] - coor_minmax_df['ymin'] - ((coor_minmax_df['ymax'] - coor_minmax_df['ymin'])/2))
    coor_df['x_2nd'] = coor_df['xnorm']**2 / 2
    coor_df['y_2nd'] = coor_df['ynorm']**2 / 2
    coor_df['x_sin'] = coor_df['xnorm'].apply(np.sin)
    coor_df['y_sin'] = coor_df['ynorm'].apply(np.sin)
    coor_df['x_cos'] = coor_df['xnorm'].apply(np.cos)
    coor_df['y_cos'] = coor_df['ynorm'].apply(np.cos)
    coor_df['xy'] = coor_df['xnorm'] * coor_df['ynorm']
    
    coor_df = coor_df.drop(columns=['x좌표', 'y좌표'])
    
    return coor_df

In [32]:
def building_age_feature_engi(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        
        if x < 0:
            return '건축전계약'
        elif x < 5:
            return '5년미만'
        elif x < 10:
            return '10년미만'
        elif x < 15:
            return '15년미만'
        elif x < 20:
            return '20년미만'
        elif x < 30:
            return '30년미만'
        elif x < 40:
            return '40년미만'
        else:
            return '40년이상'

In [33]:
def prediction_pipeline(models_list, manual_input_df, land_specs_df, land_plans_df, land_prices_df, selected_features_list, verbose=1):
    manual_input_df = manual_input_df.copy()
    land_specs_df = land_specs_df.copy()
    land_plans_df = land_plans_df.copy()
    land_prices_df = land_prices_df.copy()
    
    prediction_data_df = manual_input_df.merge(land_specs_df, on=['지번주소'], how='left')\
    .merge(land_prices_df, on=['지번주소'], how='left').merge(land_plans_df, on=['지번주소'], how='left')
    
    #prediction_data_df['건물나이'] = prediction_data_df['건물나이'].apply(building_age_feature_engi)
    
    addr_list = manual_input_df['지번주소'].unique().tolist()
    
    coor_df = fetch_coor_data(addr_list)
    coor_df = coor_df[['지번주소', 'x좌표', 'y좌표']]
    
    coor_minmax_df = pd.read_csv('./Training/training_data_ver_17/coor_minmax.csv')
    
    coor_df = coor_feature_engi(coor_df, coor_minmax_df)
    
    prediction_data_df = prediction_data_df.merge(coor_df, on=['지번주소'], how='left')
    
    prediction_data_df['건축년도'] = 2020
    
    prediction_data_df = datetime_feature_engi(prediction_data_df)
    
    '''x_cols = training_data_df.columns.drop(['주택가격지수', '가격면적', 'targetarea_log_transformed', '금액(만원)', 
                                            'target_log_transformed'
                                           ]).tolist()'''
    
    '''prediction_data_df['토지면적over총연면적times공시지가'] = (prediction_data_df['토지면적'] / prediction_data_df['총연면적'])\
    * prediction_data_df['공시지가(만원)']'''
    
    prediction_data_df = prediction_data_df[selected_features_list]
    
    cols_to_log = ['총건축면적', '총연면적', '총용적률연면적', '토지면적', '부속건축물면적']
    for col in cols_to_log:
        if col in prediction_data_df.columns.tolist():
            prediction_data_df[col] = prediction_data_df[col].apply(lambda x: np.log1p(float(x)))
        
    cat_cols_df = pd.read_csv('./Training/training_data_ver_17/cat_cols.csv')
    
    cat_cols = []
    for col in cat_cols_df['colname']:
        if col in prediction_data_df.columns:
            cat_cols.append(col)
    
    if verbose == 1:
        print('mapping categorical data...')
    
        for col in tqdm(cat_cols):
            col_map_df = pd.read_csv('./Training/training_data_ver_17/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
    
    elif verbose == 0:
        for col in cat_cols:
            col_map_df = pd.read_csv('./Training/training_data_ver_17/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
        
    
    return prediction_data_df

In [34]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 서초구 양재동 391-15'], 'trade_type':['다세대연립_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 서초구 양재동 391-15,다세대연립_매매


In [35]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)
print(prac_df.shape)
prac_df.head()

1it [00:00,  1.28it/s]
 18%|██████████████▎                                                                 | 51/286 [00:00<00:00, 509.88it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 610.97it/s]


(1, 300)


,건축년도,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type,공시지가(만원),표준지여부,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,(한강)오염행위_제한지역,(한강)폐기물매립시설_설치제한지역,4대문안,가로구역별_최고높이_제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축허가·착공제한지역,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택_공급촉진지구,공공청사,공동구,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,공항시설보호지구,과밀억제권역,광로1류(폭_70M_이상),광로2류(폭_50M~70M),광로3류(폭_40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭_35M~40M),대로2류(폭_30M~35M),대로3류(폭_25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획_입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,묘지공원,문화공원,문화시설,문화재,문화재보존영향_검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,방송통신시설,방수설비,방재지구,방화지구,변전소(전원개발사업구역),변전시설,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,소공원,소로1류(폭_10M~12M),소로2류(폭_8M~10M),소로3류(폭_8M_미만),소하천,소하천구역,수도공급시설,수평표면구역,시가지경관지구,시가지조성사업지역,시장,시장정비구역,시장정비구역기타,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,운동장,원추표면구역,원형보존지,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차전용도로,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종_구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(전술항공5km),제한보호구역(후방지역500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거용지,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭_20M~25M),중로2류(폭_15M~20M),중로3류(폭_12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역,통제보호구역(민통선이남300m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,학교,학교시설보호지구,학교이적지,학교환경위생_정화구역,현상변경허가_대상구역,홍수관리구역
0,2020,14.75,-1.0,-1.836970e-16,5029.312811,-10108.489939,1.264699e+07,5.109078e+07,0.368158,0.914884,-0.929763,0.403717,-5.083876e+07,4,610.0,0,0,1,0,2,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 68.48it/s]


In [37]:
preds_array = np.mean(preds_list, axis=0)

In [38]:
preds_array

array([1357.98872359])

In [39]:
preds_array * 554.97

array([753643.00192887])

In [41]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 강남구 개포동 1237-7'], 'trade_type':['오피스텔_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 강남구 개포동 1237-7,오피스텔_매매


In [42]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:01,  1.61s/it]
 22%|█████████████████▉                                                              | 64/286 [00:00<00:00, 633.52it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 634.00it/s]


In [43]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 40.64it/s]


In [44]:
preds = np.mean(preds_list)
preds

1004.9563360635136

In [45]:
preds1 = (preds * 29.7) + (preds * (29.99*2))
preds1

90124.4842181759

In [46]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 강남구 개포동 1237-7'], 'trade_type':['다세대연립_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 강남구 개포동 1237-7,다세대연립_매매


In [47]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:00,  2.40it/s]
 21%|████████████████▌                                                               | 59/286 [00:00<00:00, 589.86it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 631.20it/s]


In [48]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 79.35it/s]


In [49]:
preds = np.mean(preds_list)
preds

1318.0375400977198

In [50]:
preds2 = (preds * (566.12 - (29.7 + (29.99 * 2))))
preds2

627965.8056041576

In [51]:
preds1 + preds2

718090.2898223335

In [52]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 금천구 독산동 330-13'], 'trade_type':['오피스텔_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 금천구 독산동 330-13,오피스텔_매매


In [53]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:00,  1.51it/s]
 23%|██████████████████▏                                                             | 65/286 [00:00<00:00, 643.43it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 635.41it/s]


In [54]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 73.51it/s]


In [55]:
preds = np.mean(preds_list)
preds

813.6144948171698

In [56]:
preds * 1110.62

903616.530233845

In [57]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 금천구 독산동 335-10'], 'trade_type':['오피스텔_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 금천구 독산동 335-10,오피스텔_매매


In [58]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:00,  2.49it/s]
 20%|████████████████▏                                                               | 58/286 [00:00<00:00, 579.87it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 622.95it/s]


In [59]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 78.11it/s]


In [60]:
preds = np.mean(preds_list)
preds

830.4433410694415

In [61]:
preds * 1624.86

1349354.1671700927

In [62]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 구로구 구로동 481-6'], 'trade_type':['오피스텔_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 구로구 구로동 481-6,오피스텔_매매


In [63]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:01,  1.27s/it]
 20%|████████████████▏                                                               | 58/286 [00:00<00:00, 579.87it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 604.52it/s]


In [64]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 79.35it/s]


In [65]:
preds1 = np.mean(preds_list)
preds1

736.2351937947442

In [66]:
manual_input_df = pd.DataFrame({'지번주소':['서울특별시 구로구 구로동 481-6'], 'trade_type':['다세대연립_매매']})
manual_input_df

,지번주소,trade_type
0,서울특별시 구로구 구로동 481-6,다세대연립_매매


In [67]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features)

1it [00:00,  1.08it/s]
 27%|█████████████████████▎                                                          | 76/286 [00:00<00:00, 759.82it/s]

mapping categorical data...


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 748.52it/s]


In [68]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df)
    preds_list.append(preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 80.63it/s]


In [69]:
preds2 = np.mean(preds_list)
preds2

830.6874681317713

In [70]:
(preds1 * (62.52 + 62.38 + 68.37)) + (preds2 * (1340.1 - (62.52 + 62.38 + 68.37)))

1094949.4849822696

In [71]:
reference_df = pd.read_excel('./Production_Prep/이승준_총괄표.xlsx', header=1)
print(reference_df.shape)
reference_df.head()

(137, 39)


,지역,소재지,지번,용도지역,도로너비,토지면적(㎡),토지매매금액,토지매매단가\n(원/㎡),공시지가와격차\n(개공/매매),건물용도,공급면적(㎡),전유면적(㎡),전용율,용적율\n(%),건폐율\n(%),구분건물호,건물동수,건물층수,의뢰인,기준시점,전체평가금액,투입비용합계,순이익,순이익율,전유면적,공급면적,전유면적기준,공급면적기준,토지계약일자,허가일자,착공일자,사용승인일,Unnamed: 32,토지원가비율,건축원가비율,부대비용,금융비용,합계,Unnamed: 38
0,강동구,천호동,17-4외,2종일주,4~6미터,934.0,5.549000e+09,5941113.0,0.541,도시형생활주택,1720.57,1418.72,0.825,184.22,55.98,27개호,3개동,지상5층,SH공사,2019.12.18,1.005050e+10,9.128694e+09,921806160.0,0.092,7084203.0,5.841088e+06,6.434458e+06,5515021.0,2019.04.04,2019.11.26,2019.12.26,2020.07.09,5500000.0,0.6473,0.3233,0.0091,0.0203,1.0,NaN
1,강동구,천호동,18-65외,2종일반주거,6미터,261.0,1.880000e+09,7203065.0,0.484,도시형생활주택,517.56,414.78,0.689,199.83,53.57,15개호,1개동,지하1\n지상6,LH공사,2019.06.07,3.633000e+09,3.109518e+09,523482390.0,0.144,8758860.0,6.034783e+06,7.496788e+06,5932890.0,2018.06.13,2018.09.18,2018.10.19,2019.04.11,5400000.0,0.6423,0.3262,0.0091,0.0224,1.0,NaN
2,강동구,천호동,18-163외,2종일반주거,4미터,228.0,1.510000e+09,6622807.0,0.490,도시형생활주택,454.94,378.30,0.832,199.54,58.35,14세대,1개동,14세대,SH공사,2020.11.23,3.091500e+09,2.448262e+09,643238380.0,0.208,8172086.0,6.795402e+06,6.471746e+06,5588504.0,2019.09.18,2020.04.24,2020.05.21,2020.11.11,5400000.0,0.6552,0.3131,0.0088,0.0229,1.0,NaN
3,강동구,천호동,36-8외,2종일주,4미터미만,267.4,1.860000e+09,6955871.0,0.530,다세대주택,534.75,467.37,0.874,199.54,58.35,10세대,1개동,10세대,SH공사,2020.10.23,3.232500e+09,2.992845e+09,239655010.0,0.074,6916362.0,6.044881e+06,6.403588e+06,5827722.0,2019.10.30,2020.02.10,2020.03.30,2020.10.08,5500000.0,0.6612,0.3065,0.0086,0.0237,1.0,NaN
4,강동구,암사동,433-69외,3종일주,4미터,258.0,1.643000e+09,6368217.0,0.479,다세대주택,493.67,408.01,0.850,209.57,58.46,9세대,1개동,지상5층,SH공사,2020.10.23,2.901000e+09,2.657727e+09,243272910.0,0.084,7110120.0,6.040855e+06,6.513877e+06,5319617.0,2019.11.11,2020.02.14,2020.05.04,2020.09.28,5500000.0,0.6573,0.3100,0.0087,0.0241,1.0,NaN


In [72]:
reference_df['전용면적단가'] = reference_df['전체평가금액'] / reference_df['전유면적(㎡)']

In [73]:
def jibun_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        x = x.replace('외', '')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [74]:
reference_df['지번주소'] = '서울특별시 ' + reference_df['지역'] + ' ' + reference_df['소재지'] + ' ' + reference_df['지번'].apply(jibun_modifier)

In [75]:
def assign_tradetype(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if '오피' in x:
            return '오피스텔_매매'
        else:
            return '다세대연립_매매'

In [76]:
reference_df['trade_type'] = reference_df['건물용도'].apply(assign_tradetype)

In [77]:
reference_df['trade_type'].value_counts()

다세대연립_매매    130
오피스텔_매매       1
Name: trade_type, dtype: int64

In [78]:
reference_df['예측전용면적단가'] = np.nan

In [79]:
for i in tqdm(range(reference_df.shape[0])):
    addr = reference_df.loc[i, '지번주소']
    ttype = reference_df.loc[i, 'trade_type']
    
    if (pd.isna(addr) == True) | (pd.isna(ttype) == True):
        continue
    
    manual_input_df = pd.DataFrame({'지번주소':[addr], 'trade_type':[ttype]})
    
    prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features, verbose=0)
    
    if prac_df.isna().sum().sum() > 0:
        continue
    
    preds_list = []
    for model in models_list:
        preds = model.predict(prac_df)
        preds_list.append(preds)
        
    reference_df.loc[i, '예측전용면적단가'] = np.mean(preds_list)

  0%|                                                                                          | 0/137 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.14s/it]
  1%|▌                                                                                 | 1/137 [00:10<22:57, 10.13s/it]
0it [00:00, ?it/s]
1it [00:00,  1.24it/s]
  1%|█▏                                                                                | 2/137 [00:18<20:05,  8.93s/it]
0it [00:00, ?it/s]
1it [00:00,  1.13it/s]
  2%|█▊                                                                                | 3/137 [00:26<19:07,  8.56s/it]
0it [00:00, ?it/s]
1it [00:00,  2.65it/s]
  3%|██▍                                                                               | 4/137 [00:33<18:10,  8.20s/it]
0it [00:00, ?it/s]
1it [00:00,  2.49it/s]
  4%|██▉                                                                               | 5/137 [00:41<17:37,  8.01s/it]
0it [00:00, ?it/s]
1it [00:00,  3.48it/s]
  4%|███▌                   

0it [00:00, ?it/s]
1it [00:00,  1.19it/s]
 36%|████████████████████████████▉                                                    | 49/137 [06:19<11:09,  7.61s/it]
0it [00:00, ?it/s]
1it [00:00,  1.57it/s]
 36%|█████████████████████████████▌                                                   | 50/137 [06:27<11:02,  7.61s/it]
0it [00:00, ?it/s]
1it [00:00,  3.44it/s]
 37%|██████████████████████████████▏                                                  | 51/137 [06:34<10:46,  7.52s/it]
0it [00:00, ?it/s]
1it [00:00,  3.51it/s]
 38%|██████████████████████████████▋                                                  | 52/137 [06:41<10:34,  7.47s/it]
0it [00:00, ?it/s]
1it [00:00,  2.10it/s]
 39%|███████████████████████████████▎                                                 | 53/137 [06:49<10:33,  7.54s/it]
0it [00:00, ?it/s]
1it [00:00,  2.91it/s]
 39%|███████████████████████████████▉                                                 | 54/137 [06:57<10:21,  7.49s/it]
0it [00:00, ?it/s]
1it [00:0

 71%|█████████████████████████████████████████████████████████▎                       | 97/137 [13:07<05:48,  8.71s/it]
0it [00:00, ?it/s]
1it [00:00,  2.62it/s]
 72%|█████████████████████████████████████████████████████████▉                       | 98/137 [13:16<05:35,  8.61s/it]
0it [00:00, ?it/s]
1it [00:00,  1.72it/s]
 72%|██████████████████████████████████████████████████████████▌                      | 99/137 [13:24<05:29,  8.68s/it]
0it [00:00, ?it/s]
1it [00:00,  1.58it/s]
 73%|██████████████████████████████████████████████████████████▍                     | 100/137 [13:33<05:22,  8.72s/it]
0it [00:00, ?it/s]
1it [00:00,  1.46it/s]
 74%|██████████████████████████████████████████████████████████▉                     | 101/137 [13:42<05:15,  8.76s/it]
0it [00:00, ?it/s]
1it [00:00,  2.45it/s]
 74%|███████████████████████████████████████████████████████████▌                    | 102/137 [13:51<05:04,  8.70s/it]
0it [00:00, ?it/s]
1it [00:00,  3.06it/s]
 75%|███████████████████████

In [80]:
reference_df.head()

,지역,소재지,지번,용도지역,도로너비,토지면적(㎡),토지매매금액,토지매매단가\n(원/㎡),공시지가와격차\n(개공/매매),건물용도,공급면적(㎡),전유면적(㎡),전용율,용적율\n(%),건폐율\n(%),구분건물호,건물동수,건물층수,의뢰인,기준시점,전체평가금액,투입비용합계,순이익,순이익율,전유면적,공급면적,전유면적기준,공급면적기준,토지계약일자,허가일자,착공일자,사용승인일,Unnamed: 32,토지원가비율,건축원가비율,부대비용,금융비용,합계,Unnamed: 38,전용면적단가,지번주소,trade_type,예측전용면적단가
0,강동구,천호동,17-4외,2종일주,4~6미터,934.0,5.549000e+09,5941113.0,0.541,도시형생활주택,1720.57,1418.72,0.825,184.22,55.98,27개호,3개동,지상5층,SH공사,2019.12.18,1.005050e+10,9.128694e+09,921806160.0,0.092,7084203.0,5.841088e+06,6.434458e+06,5515021.0,2019.04.04,2019.11.26,2019.12.26,2020.07.09,5500000.0,0.6473,0.3233,0.0091,0.0203,1.0,NaN,7.084203e+06,서울특별시 강동구 천호동 17-4,다세대연립_매매,767.383364
1,강동구,천호동,18-65외,2종일반주거,6미터,261.0,1.880000e+09,7203065.0,0.484,도시형생활주택,517.56,414.78,0.689,199.83,53.57,15개호,1개동,지하1\n지상6,LH공사,2019.06.07,3.633000e+09,3.109518e+09,523482390.0,0.144,8758860.0,6.034783e+06,7.496788e+06,5932890.0,2018.06.13,2018.09.18,2018.10.19,2019.04.11,5400000.0,0.6423,0.3262,0.0091,0.0224,1.0,NaN,8.758860e+06,서울특별시 강동구 천호동 18-65,다세대연립_매매,889.325227
2,강동구,천호동,18-163외,2종일반주거,4미터,228.0,1.510000e+09,6622807.0,0.490,도시형생활주택,454.94,378.30,0.832,199.54,58.35,14세대,1개동,14세대,SH공사,2020.11.23,3.091500e+09,2.448262e+09,643238380.0,0.208,8172086.0,6.795402e+06,6.471746e+06,5588504.0,2019.09.18,2020.04.24,2020.05.21,2020.11.11,5400000.0,0.6552,0.3131,0.0088,0.0229,1.0,NaN,8.172086e+06,서울특별시 강동구 천호동 18-163,다세대연립_매매,887.609074
3,강동구,천호동,36-8외,2종일주,4미터미만,267.4,1.860000e+09,6955871.0,0.530,다세대주택,534.75,467.37,0.874,199.54,58.35,10세대,1개동,10세대,SH공사,2020.10.23,3.232500e+09,2.992845e+09,239655010.0,0.074,6916362.0,6.044881e+06,6.403588e+06,5827722.0,2019.10.30,2020.02.10,2020.03.30,2020.10.08,5500000.0,0.6612,0.3065,0.0086,0.0237,1.0,NaN,6.916362e+06,서울특별시 강동구 천호동 36-8,다세대연립_매매,850.561056
4,강동구,암사동,433-69외,3종일주,4미터,258.0,1.643000e+09,6368217.0,0.479,다세대주택,493.67,408.01,0.850,209.57,58.46,9세대,1개동,지상5층,SH공사,2020.10.23,2.901000e+09,2.657727e+09,243272910.0,0.084,7110120.0,6.040855e+06,6.513877e+06,5319617.0,2019.11.11,2020.02.14,2020.05.04,2020.09.28,5500000.0,0.6573,0.3100,0.0087,0.0241,1.0,NaN,7.110120e+06,서울특별시 강동구 암사동 433-69,다세대연립_매매,768.466804


In [81]:
reference_df['전용면적단가'] = reference_df['전용면적단가'] / 10000

In [82]:
reference_df.dropna(subset=['전용면적단가', '예측전용면적단가'], inplace=True)

In [84]:
from sklearn.metrics import mean_squared_error

In [85]:
np.sqrt(mean_squared_error(reference_df['전용면적단가'], reference_df['예측전용면적단가']))

105.53040788963412

In [86]:
reference_df.to_excel('./Production_Prep/이승준_총괄표_예측포함_no_landarea_도생을_다세대로.xlsx')

In [87]:
def assign_tradetype(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if '다세' in x:
            return '다세대연립_매매'
        elif '오피' in x:
            return '오피스텔_매매'
        else:
            return '아파트_매매'  

In [88]:
reference_df['trade_type'] = reference_df['건물용도'].apply(assign_tradetype)

In [89]:
reference_df['예측전용면적단가'] = np.nan

In [91]:
reference_df.reset_index(drop=True, inplace=True)

In [93]:
for i in tqdm(range(reference_df.shape[0])):
    addr = reference_df.loc[i, '지번주소']
    ttype = reference_df.loc[i, 'trade_type']
    
    if (pd.isna(addr) == True) | (pd.isna(ttype) == True):
        continue
    
    manual_input_df = pd.DataFrame({'지번주소':[addr], 'trade_type':[ttype]})
    
    prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features, verbose=0)
    
    if prac_df.isna().sum().sum() > 0:
        continue
    
    preds_list = []
    for model in models_list:
        preds = model.predict(prac_df)
        preds_list.append(preds)
        
    reference_df.loc[i, '예측전용면적단가'] = np.mean(preds_list)

  0%|                                                                                          | 0/128 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.46it/s]
  1%|▋                                                                                 | 1/128 [00:09<21:03,  9.95s/it]
0it [00:00, ?it/s]
1it [00:00,  4.29it/s]
  2%|█▎                                                                                | 2/128 [00:18<19:17,  9.19s/it]
0it [00:00, ?it/s]
1it [00:00,  3.71it/s]
  2%|█▉                                                                                | 3/128 [00:26<17:49,  8.56s/it]
0it [00:00, ?it/s]
1it [00:00,  3.35it/s]
  3%|██▌                                                                               | 4/128 [00:34<17:24,  8.43s/it]
0it [00:00, ?it/s]
1it [00:00,  3.04it/s]
  4%|███▏                                                                              | 5/128 [00:42<17:07,  8.36s/it]
0it [00:00, ?it/s]
1it [00:00,  4.15it/s]
  5%|███▊                   

0it [00:00, ?it/s]
1it [00:00,  3.98it/s]
 38%|███████████████████████████████                                                  | 49/128 [06:49<10:56,  8.31s/it]
0it [00:00, ?it/s]
1it [00:00,  3.42it/s]
 39%|███████████████████████████████▋                                                 | 50/128 [06:58<11:01,  8.48s/it]
0it [00:00, ?it/s]
1it [00:00,  2.46it/s]
 40%|████████████████████████████████▎                                                | 51/128 [07:07<10:50,  8.44s/it]
0it [00:00, ?it/s]
1it [00:00,  3.40it/s]
 41%|████████████████████████████████▉                                                | 52/128 [07:15<10:36,  8.37s/it]
0it [00:00, ?it/s]
1it [00:00,  3.98it/s]
 41%|█████████████████████████████████▌                                               | 53/128 [07:23<10:22,  8.30s/it]
0it [00:00, ?it/s]
1it [00:00,  3.80it/s]
 42%|██████████████████████████████████▏                                              | 54/128 [07:31<10:10,  8.26s/it]
0it [00:00, ?it/s]
1it [00:0

 76%|█████████████████████████████████████████████████████████████▍                   | 97/128 [13:25<04:16,  8.28s/it]
0it [00:00, ?it/s]
1it [00:00,  3.93it/s]
 77%|██████████████████████████████████████████████████████████████                   | 98/128 [13:33<04:09,  8.33s/it]
0it [00:00, ?it/s]
1it [00:00,  3.46it/s]
 77%|██████████████████████████████████████████████████████████████▋                  | 99/128 [13:41<04:00,  8.30s/it]
0it [00:00, ?it/s]
1it [00:00,  3.89it/s]
 78%|██████████████████████████████████████████████████████████████▌                 | 100/128 [13:50<03:53,  8.34s/it]
0it [00:00, ?it/s]
1it [00:00,  4.06it/s]
 79%|███████████████████████████████████████████████████████████████▏                | 101/128 [13:58<03:43,  8.27s/it]
0it [00:00, ?it/s]
1it [00:00,  3.16it/s]
 80%|███████████████████████████████████████████████████████████████▊                | 102/128 [14:06<03:33,  8.22s/it]
0it [00:00, ?it/s]
1it [00:00,  4.19it/s]
 80%|███████████████████████

In [94]:
np.sqrt(mean_squared_error(reference_df['전용면적단가'], reference_df['예측전용면적단가']))

108.20872286917158

In [95]:
reference_df.to_excel('./Production_Prep/이승준_총괄표_예측포함_no_landarea_도생을_아파트로.xlsx')

In [ ]:
features_to_fill_df = pd.read_excel('./Production_Prep/직접작성필요목록_독산동33013.xlsx')
transposed_df = features_to_fill_df.T
manual_input_df = transposed_df.iloc[2:, :]
manual_input_df.columns = transposed_df.loc['features',:]
manual_input_df.reset_index(drop=True, inplace=True)
manual_input_df.dropna(inplace=True)
print(manual_input_df.shape)
manual_input_df.head()

In [ ]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, selected_features_list)

In [ ]:
prac_df.shape

In [ ]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df.iloc[:,:22])
    preds_list.append(preds)

In [ ]:
prac_df['preds'] = np.mean(preds_list, axis=0)
prac_df['preds'] = np.e ** prac_df['preds']

In [ ]:
prac_df['가격'] = prac_df['전용면적'] * prac_df['preds']

In [ ]:
prac_df['가격'].sum()

In [ ]:
preds_list = []
for model in tqdm(models_list2):
    preds = model.predict(prac_df.iloc[:,:22])
    preds_list.append(preds)

In [ ]:
prac_df['preds'] = np.mean(preds_list, axis=0)
prac_df['preds'] = np.e ** prac_df['preds']

In [ ]:
prac_df['가격'] = prac_df['전용면적'] * prac_df['preds']

In [ ]:
prac_df['가격'].sum()

In [ ]:
features_to_fill_df = pd.read_excel('./Production_Prep/직접작성필요목록_독산동33510.xlsx')
transposed_df = features_to_fill_df.T
manual_input_df = transposed_df.iloc[2:, :]
manual_input_df.columns = transposed_df.loc['features',:]
manual_input_df.reset_index(drop=True, inplace=True)
manual_input_df.dropna(inplace=True)
print(manual_input_df.shape)
manual_input_df.head()

In [ ]:
prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, selected_features_list)

In [ ]:
prac_df.shape

In [ ]:
preds_list = []
for model in tqdm(models_list):
    preds = model.predict(prac_df.iloc[:,:22])
    preds_list.append(preds)

In [ ]:
prac_df['preds'] = np.mean(preds_list, axis=0)

In [ ]:
prac_df['가격'] = prac_df['전용면적'] * prac_df['preds']

In [ ]:
prac_df

In [ ]:
(prac_df.iloc[0,-1] * 14) + (prac_df.iloc[1,-1] * 14) + (prac_df.iloc[2, -1] * 6)

In [ ]:
basedir = './prepped_data/'

In [ ]:
%%time
main_data_df = pd.read_csv(basedir + 'main_data_monthly_converted.csv')
print(main_data_df.shape)
main_data_df.head()

In [ ]:
%%time
land_prices_df = pd.read_csv(basedir + 'land_prices_ver_2.csv')
land_prices_df.shape

In [ ]:
%%time
land_specs_df = pd.read_csv(basedir + 'land_specs_ver_3.csv')
land_specs_df.shape

In [ ]:
%%time
land_plans_df = pd.read_csv(basedir + 'land_plans_ver_3.csv')
land_plans_df.shape

In [ ]:
land_plans_df.fillna('해당사항없음', inplace=True)

In [ ]:
%%time
bdinfo_df = pd.read_csv(basedir + 'bdinfo_features_structure_purpose_roof.csv')
bdinfo_df.shape

In [ ]:
%%time
bdinfo2_df = pd.read_csv(basedir + 'new_building_features_2021_0209.csv')
print(bdinfo2_df.shape)
bdinfo2_df.head()

In [ ]:
main_data_df.rename(columns={'전용면적(㎡)':'전용면적'}, inplace=True)

In [ ]:
%%time
main_data_df['전용면적_mean'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('mean')['전용면적']
main_data_df['전용면적_min'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('min')['전용면적']
main_data_df['전용면적_max'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('max')['전용면적']

In [ ]:
main_data_df['가격/면적'] = main_data_df['금액(만원)'] / main_data_df['전용면적']

In [ ]:
main_data_df['건물나이'] = main_data_df['년'] - main_data_df['건축년도']
main_data_df['건물나이'].describe()

In [ ]:
main_data_df['year_linear'] = (main_data_df['년'] - main_data_df['년'].min()) + ((main_data_df['월']-1)/12)

In [ ]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [ ]:
main_data_df['month_sin'] = main_data_df['월'].apply(month_circular_sine_func)
main_data_df['month_cos'] = main_data_df['월'].apply(month_circular_cosine_func)

In [ ]:
main_data_df.drop(columns=['월', '계약일'], inplace=True)

In [ ]:
xmax = main_data_df['X좌표'].max()
xmin = main_data_df['X좌표'].min()
ymax = main_data_df['Y좌표'].max()
ymin = main_data_df['Y좌표'].min()

In [ ]:
coor_minmax_df = pd.DataFrame({'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, index=[0])
coor_minmax_df.to_csv('./Training/training_data_ver_14/coor_minmax.csv', index=False)
coor_minmax_df

In [ ]:
main_data_df['xnorm'] = (main_data_df['X좌표'] - xmin - ((xmax - xmin)/2))
main_data_df['ynorm'] = (main_data_df['Y좌표'] - ymin - ((ymax - ymin)/2))
main_data_df['x_2nd'] = main_data_df['xnorm']**2 / 2
main_data_df['y_2nd'] = main_data_df['ynorm']**2 / 2
main_data_df['x_sin'] = main_data_df['xnorm'].apply(np.sin)
main_data_df['y_sin'] = main_data_df['ynorm'].apply(np.sin)
main_data_df['x_cos'] = main_data_df['xnorm'].apply(np.cos)
main_data_df['y_cos'] = main_data_df['ynorm'].apply(np.cos)
main_data_df['xy'] = main_data_df['xnorm'] * main_data_df['ynorm']

In [ ]:
main_data_df.drop(columns=['X좌표', 'Y좌표'], inplace=True)

In [ ]:
main_data_df['trade_type'] = main_data_df['건물종류'] + '_' + main_data_df['전월세매매구분']
main_data_df.drop(columns=['건물종류', '전월세매매구분'], inplace=True)

In [ ]:
merge1_df = main_data_df.merge(land_prices_df, on=['지번주소', '년'])
merge1_df.shape

In [ ]:
merge1_df['공시지가'] = merge1_df['공시지가'] / 10000
merge1_df.rename(columns={'공시지가':'공시지가(만원)'}, inplace=True)

In [ ]:
bdinfo_df.drop(columns=['사용승인년도'], inplace=True)
bdinfo_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
merge1_df.shape

In [ ]:
merge2_df = merge1_df.merge(bdinfo_df, on=['지번주소', '년'])
merge2_df.shape

In [ ]:
%%time
bdinfo3_df = pd.read_csv(basedir + 'extracted_building_features.csv')
bdinfo3_df = bdinfo3_df[['지번주소', '기준년도'] + [col for col in bdinfo3_df.columns if '층' in col]]
print(bdinfo3_df.shape)
bdinfo3_df.head()

In [ ]:
bdinfo3_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
bdinfo2_df.head()

In [ ]:
bdinfo2_df.drop(columns=['사용승인년도', '대장종류'], inplace=True)

In [ ]:
bdinfo2_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
to_drop_df = bdinfo2_df[(bdinfo2_df['부속건축물수'] > 0) & (bdinfo2_df['부속건축물면적'] == 0)]
print(to_drop_df.shape)

In [ ]:
bdinfo2_df.drop(to_drop_df.index, inplace=True)

In [ ]:
merge2_df = merge2_df.merge(bdinfo2_df, on=['지번주소', '년'])
print(merge2_df.shape)

In [ ]:
merge2_df = merge2_df.merge(bdinfo3_df, on=['지번주소', '년'])

In [ ]:
merge2_df.head()

In [ ]:
merge2_df['최고층-층'] = merge2_df['층max'] - merge2_df['층']
merge2_df['최고층-층'].describe()

In [ ]:
negative_df = merge2_df[merge2_df['최고층-층'] < 0]
negative_df.shape

In [ ]:
negative_df['지번주소'].nunique()

In [ ]:
idx_to_drop = []
for addr in tqdm(negative_df['지번주소'].unique()):
    addr_df = merge2_df[merge2_df['지번주소'] == addr]
    addr2_df = negative_df[negative_df['지번주소'] == addr]
    min_year = addr2_df['년'].min()
    df_to_drop = addr_df[addr_df['년'] >= min_year]
    for idx in df_to_drop.index:
        idx_to_drop.append(idx)

In [ ]:
len(list(set(idx_to_drop)))

In [ ]:
merge2_df.drop(idx_to_drop, inplace=True)

In [ ]:
merge2_df.shape

In [ ]:
merge2_df['층norm'] = merge2_df['층'] / merge2_df['층max']

In [ ]:
merge2_df['층norm'].describe()

In [ ]:
merge2_df.drop(columns=['최고층-층'], inplace=True)

In [ ]:
merge2_df['지하세대있음'] = merge2_df['층'].apply(lambda x: 1 if x < 0 else 0)
merge2_df['지하세대있음'].value_counts()

In [ ]:
cols = [col for col in land_specs_df.columns if col != '공시지가']
cols

In [ ]:
merge2_df = merge2_df.merge(land_specs_df[cols], on=['지번주소', '년'])
merge2_df.shape

In [ ]:
merge2_df.drop(columns=['대지면적'], inplace=True)

In [ ]:
merge3_df = merge2_df.merge(land_plans_df, on=['지번주소', '년'])
merge3_df.shape

In [ ]:
merge3_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in merge3_df.columns]

In [ ]:
land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in land_plans_df.columns]

In [ ]:
lp_catcols = land_plans_df.columns.tolist()[2:]
lp_catcols[:5]

In [ ]:
ls_catcols = [col for col in cols if col not in ['지번주소', '년', '토지면적']]
ls_catcols

In [ ]:
def building_age_feature_engi(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        
        if x < 0:
            return '건축전계약'
        elif x < 5:
            return '5년미만'
        elif x < 10:
            return '10년미만'
        elif x < 15:
            return '15년미만'
        elif x < 20:
            return '20년미만'
        elif x < 30:
            return '30년미만'
        elif x < 40:
            return '40년미만'
        else:
            return '40년이상'

In [ ]:
merge3_df = merge3_df[merge3_df['건물나이'] >= 0]
merge3_df.shape

In [ ]:
merge3_df['건물나이'] = merge3_df['건물나이'].apply(building_age_feature_engi)
merge3_df['건물나이'].value_counts()

In [ ]:
merge3_df.head()

In [ ]:
merge3_df.drop(columns=['년', '도로명주소'], inplace=True)

In [ ]:
cat_cols = ['trade_type', '표준지여부', '건물나이'] + ls_catcols + lp_catcols

In [ ]:
for col in tqdm(cat_cols):
    col_map = {}
    col_map_df = pd.DataFrame({'cat':merge3_df[col].unique(),'map':[i for i in range(len(merge3_df[col].unique()))]})
    for i in range(col_map_df.shape[0]):
        col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
    merge3_df[col] = merge3_df[col].map(col_map)
    col_map_df.to_csv('./Training/training_data_ver_16/feature_maps/{}_map.csv'.format(col))

In [ ]:
cat_cols_df = pd.DataFrame({'colname':cat_cols})
cat_cols_df.head()

In [ ]:
cat_cols_df.to_csv('./Training/training_data_ver_16/cat_cols.csv', index=False)

In [ ]:
cols_to_drop = []
for col in tqdm(merge3_df.columns):
    try:
        if merge3_df[col].std() == 0:
            cols_to_drop.append(col)
        else:
            pass
    except:
        pass

In [ ]:
len(cols_to_drop)

In [ ]:
cols_to_drop

In [ ]:
merge3_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
%%time
merge3_df.to_csv('./Training/training_data_ver_16/training_data_ver_16.csv', index=False)

In [ ]:
merge3_df.shape